### XXXX-2022: END SEMESTER ASSESSMENT (ESA) 
## M TECH DATA SCIENCE AND MACHINE LEARNING_ SEMESTER II

#### UE20CS933 - NATURAL LANGUAGE PROCESSING 

### Imports

In [10]:
import pandas as pd
import numpy as np
import numpy as np
from nltk.corpus import stopwords 
from gensim.parsing.preprocessing import PorterStemmer, remove_stopwords
import string 
import unicodedata
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import re

## Section B
### 2. Use the data.csv dataset as provided in the notebook as Pandas DataFrame and   process it as questioned below.

#### Dataset 
Airline Sentiment dataset is provide as pandas dataframe. Using python NLP libraires process the dataset as questioned below.

In [2]:
import pandas as pd
data = pd.read_csv('data.csv')
data.columns

Index(['tweet_id', 'airline_sentiment', 'airline_sentiment_confidence',
       'negativereason', 'negativereason_confidence', 'airline',
       'airline_sentiment_gold', 'name', 'negativereason_gold',
       'retweet_count', 'text', 'tweet_coord', 'tweet_created',
       'tweet_location', 'user_timezone'],
      dtype='object')

### 2.a. Create a new Pandas DataFrame by fetching two columns 'text’ and 'airline_sentiment' from data.csv.  (Marks-6)

In [5]:
df = data[['text','airline_sentiment']].copy(deep=True)
df

,text,airline_sentiment
0,@VirginAmerica What @dhepburn said.,neutral
1,@VirginAmerica plus you've added commercials t...,positive
2,@VirginAmerica I didn't today... Must mean I n...,neutral
3,@VirginAmerica it's really aggressive to blast...,negative
4,@VirginAmerica and it's a really big bad thing...,negative
...,...,...
14635,@AmericanAir thank you we got on a different f...,positive
14636,@AmericanAir leaving over 20 minutes Late Flig...,negative
14637,@AmericanAir Please bring American Airlines to...,neutral
14638,"@AmericanAir you have my money, you change my ...",negative


### 2.b. Clean the 'text' columns as questioned below. 


#### 2.b.(i) Convert all text to lower case ( Marks- 4) 

In [11]:
def lower_text(text):
    return text.lower()

df['cleaned_text'] = df['text'].apply(lower_text)
df['cleaned_text']

0                      @virginamerica what @dhepburn said.
1        @virginamerica plus you've added commercials t...
2        @virginamerica i didn't today... must mean i n...
3        @virginamerica it's really aggressive to blast...
4        @virginamerica and it's a really big bad thing...
                               ...                        
14635    @americanair thank you we got on a different f...
14636    @americanair leaving over 20 minutes late flig...
14637    @americanair please bring american airlines to...
14638    @americanair you have my money, you change my ...
14639    @americanair we have 8 ppl so we need 2 know h...
Name: cleaned_text, Length: 14640, dtype: object

#### 2.b.(ii)  Remove the URLs (http & www) from text ( Marks-6)

In [15]:
# hint: regular expression for URLs  matching is 'https?://\S+|www\.\S+' 
def remove_http_www(text):
    return re.sub(r'https?://\S+|www\.\S+', '', text)

df['cleaned_text'] = df['cleaned_text'].apply(remove_http_www)
df['cleaned_text']

0                      @virginamerica what @dhepburn said.
1        @virginamerica plus you've added commercials t...
2        @virginamerica i didn't today... must mean i n...
3        @virginamerica it's really aggressive to blast...
4        @virginamerica and it's a really big bad thing...
                               ...                        
14635    @americanair thank you we got on a different f...
14636    @americanair leaving over 20 minutes late flig...
14637    @americanair please bring american airlines to...
14638    @americanair you have my money, you change my ...
14639    @americanair we have 8 ppl so we need 2 know h...
Name: cleaned_text, Length: 14640, dtype: object

#### 2.b.(iii).  Remove stopwords from text.  ( Marks-8)

In [24]:
# any of nltk or gensim can be used
def drop_stop_words(text):
    engstop_words = stopwords.words('english')
    tokens = text.split()
    tokens = [x for x in tokens if x not in engstop_words]
    return " ".join(tokens)

drop_stop_words("I am working with your student to train all the batches on python")

df['cleaned_text'] = df['cleaned_text'].apply(drop_stop_words)

#### 2.b.(iv) Remove punctuations from text. (Marks-8)

In [26]:
# hint: refer string.punctuation to fetch all punctuations
text = "I want to remove these '!\"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~' and these '!\"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'"

def drop_punctuations(text):
    return re.sub(r"[^\w\s]", '', text)

df['cleaned_text'] = df['cleaned_text'].apply(drop_punctuations)
df['cleaned_text']

0                              virginamerica dhepburn said
1        virginamerica plus added commercials experienc...
2        virginamerica today must mean need take anothe...
3        virginamerica really aggressive blast obnoxiou...
4                       virginamerica really big bad thing
                               ...                        
14635       americanair thank got different flight chicago
14636    americanair leaving 20 minutes late flight war...
14637    americanair please bring american airlines bla...
14638    americanair money change flight answer phones ...
14639    americanair 8 ppl need 2 know many seats next ...
Name: cleaned_text, Length: 14640, dtype: object

### 2.c. Fetch the top six most frequently used words from the text corpus (Marks-8)

In [29]:
words_list = list()
for sentence in df['cleaned_text'].values:
    tokens = sentence.split()
    for token in tokens:
        words_list.append(token)

df_words_list = pd.DataFrame(data={'words':words_list})
df_words_list.groupby(by='words')['words'].agg(['count']).sort_values(by='count',ascending=False).head(10)

,count
words,
united,4143
flight,3873
usairways,3051
americanair,2957
southwestair,2452
jetblue,2361
get,1334
thanks,1072
cancelled,1056


### 3. Use the cleaned dataframe from previous section inorder to build ML model as questioned below.

### 3.a. Convert the text feature/column into numerical using Count-vectorization  (Marks-7)

In [33]:
# hint: use below count_vectorize  method definition
count_vectorizer = CountVectorizer(analyzer='word', ngram_range=(1, 1))
cvectors = count_vectorizer.fit_transform(df['cleaned_text'])
cvectors.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

### 3.b. Convert the text  feature/column into numerical using TF-IDF (Marks-7)

In [32]:
# hint: use below TfidfVectorizer method definition
tfidf_vectorizer = TfidfVectorizer(analyzer='word',stop_words='english', ngram_range=(1, 1))
tfidf_vectors = tfidf_vectorizer.fit_transform(df['cleaned_text'])
tfidf_vectors.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

### 3.c Split both Count-vectorirsed  & TF-IDF dataset into train & test set  with one fourth records being held for testing also ensure stratified sampling of traget i.e. airline_sentiment on both split ( Marks-6)

In [35]:
# hint  use the split method : train_test_split(vectorise_dataframe,Y,stratify=Y,test_size= )
df.groupby(by='airline_sentiment')['airline_sentiment'].agg(['count'])

,count
airline_sentiment,
negative,9178
neutral,3099
positive,2363


In [38]:
df.loc[df['airline_sentiment'] == 'positive', 'airline_sentiment' ] = 3
df.loc[df['airline_sentiment'] == 'neutral', 'airline_sentiment' ]  = 2
df.loc[df['airline_sentiment'] == 'negative', 'airline_sentiment' ] = 1
df['airline_sentiment'] = df['airline_sentiment'].astype('int32')
df

,text,airline_sentiment,cleaned_text
0,@VirginAmerica What @dhepburn said.,2,virginamerica dhepburn said
1,@VirginAmerica plus you've added commercials t...,3,virginamerica plus added commercials experienc...
2,@VirginAmerica I didn't today... Must mean I n...,2,virginamerica today must mean need take anothe...
3,@VirginAmerica it's really aggressive to blast...,1,virginamerica really aggressive blast obnoxiou...
4,@VirginAmerica and it's a really big bad thing...,1,virginamerica really big bad thing
...,...,...,...
14635,@AmericanAir thank you we got on a different f...,3,americanair thank got different flight chicago
14636,@AmericanAir leaving over 20 minutes Late Flig...,1,americanair leaving 20 minutes late flight war...
14637,@AmericanAir Please bring American Airlines to...,2,americanair please bring american airlines bla...
14638,"@AmericanAir you have my money, you change my ...",1,americanair money change flight answer phones ...


In [63]:
# split count-vectorised dataset
# 2 marks
Xcnt = cvectors
ycnt = df['airline_sentiment']
Xcnt_train, Xcnt_test, ycnt_train, ycnt_test = train_test_split(Xcnt, ycnt, test_size=0.25, random_state=43)
Xcnt_train.shape, Xcnt_test.shape, ycnt_train.shape, ycnt_test.shape

((10980, 15473), (3660, 15473), (10980,), (3660,))

In [64]:
# split tf-idf dataset
#2 marks
Xtfidf = tfidf_vectors
ytfidf = df['airline_sentiment']
Xtfidf_train, Xtfidf_test, ytfidf_train, ytfidf_test = train_test_split(Xtfidf, ytfidf, test_size=0.25, random_state=43)
Xtfidf_train.shape, Xtfidf_test.shape, ytfidf_train.shape, ytfidf_test.shape

((10980, 15203), (3660, 15203), (10980,), (3660,))

### 3.d Build a basic logistic regression model ( LogisticRegression(solver='liblinear')) on Count-vectorize train set and find out its accuracy on Count-vectorize test set.  ( Marks-10 = 5 -training + 5-finding accuracy )


In [81]:
#hint:  use below LogisticRegression method
lrcnt = LogisticRegression(solver='liblinear')
lrcnt.fit(Xcnt_train, ycnt_train)
# train lr on Count-vectorize trainset
ycnt_pred = lrcnt.predict(Xcnt_test)
# find accuracy on Count-vectorize test set 
accuracy_score(ycnt_test, ycnt_pred)

0.7890710382513662

In [82]:
ycnt_pred

array([2, 1, 1, ..., 1, 3, 3], dtype=int32)

In [83]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, auc, roc_auc_score

In [84]:
rpt = classification_report(ycnt_test, ycnt_pred)
print(rpt)

              precision    recall  f1-score   support

           1       0.84      0.90      0.87      2284
           2       0.63      0.54      0.58       774
           3       0.77      0.67      0.72       602

    accuracy                           0.79      3660
   macro avg       0.75      0.71      0.72      3660
weighted avg       0.78      0.79      0.78      3660



In [85]:
confusion_matrix(ycnt_test, ycnt_pred)

array([[2061,  171,   52],
       [ 284,  421,   69],
       [ 122,   74,  406]])

### 3.e Build a basic logistic regression model ( LogisticRegression(solver='liblinear')) on TF-IDF train set and find out its accuracy on TF-IDF test set.  Which model has better accuracy ?(  Marks-10 = 5(training) + 5 (finding accuracy) )


In [86]:
 # train lr on TF-IDF train set
lrtfidf = LogisticRegression(solver='liblinear')
lrtfidf.fit(Xtfidf_train, ytfidf_train)
# train lr on Count-vectorize trainset
ytfidf_pred = lrtfidf.predict(Xtfidf_test)
# find accuracy on Count-vectorize test set 
accuracy_score(ytfidf_test, ytfidf_pred)
# find accuracy on TF-IDF  test set 

0.7669398907103825

In [87]:
rpt = classification_report(ytfidf_test, ytfidf_pred)
print(rpt)

              precision    recall  f1-score   support

           1       0.77      0.96      0.85      2284
           2       0.70      0.35      0.47       774
           3       0.83      0.57      0.68       602

    accuracy                           0.77      3660
   macro avg       0.77      0.63      0.67      3660
weighted avg       0.76      0.77      0.74      3660



In [88]:
rpt = classification_report(ytfidf_test, ytfidf_pred)
print(rpt)

              precision    recall  f1-score   support

           1       0.77      0.96      0.85      2284
           2       0.70      0.35      0.47       774
           3       0.83      0.57      0.68       602

    accuracy                           0.77      3660
   macro avg       0.77      0.63      0.67      3660
weighted avg       0.76      0.77      0.74      3660



In [89]:
confusion_matrix(ytfidf_test, ytfidf_pred)

array([[2190,   63,   31],
       [ 462,  274,   38],
       [ 203,   56,  343]])

# CountVectorizer based model is performing compared to tfidf model for classes 1 and 3.